# Integração Google Sheets -> Python -> MySQL
O objetivo deste notebook é aprender a pegar informações de planilhas sheets, e, utilizando python, passar para o MySQL

In [19]:
def main():
    """Shows basic usage of the Sheets API.
    Prints values from a sample spreadsheet.
    """
    creds = None
    if os.path.exists("token.json"):
        creds = Credentials.from_authorized_user_file("token.json", SCOPES)
    if not creds or not creds.valid:
        if creds and creds.expired and creds.refresh_token:
            creds.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                "client_secret_1003057466815-eri6a3v0j1ktaj6983mnjr80slp8h3je.apps.googleusercontent.com.json", SCOPES
            )
            creds = flow.run_local_server(port=0)
        with open("token.json", "w") as token:
            token.write(creds.to_json())

    try:
        service = build("sheets", "v4", credentials=creds)

        # Primeiro, obtenha as informações da planilha
        sheet_metadata = service.spreadsheets().get(spreadsheetId=SAMPLE_SPREADSHEET_ID).execute()
        sheet = sheet_metadata['sheets'][0]  # Pega a primeira aba
        row_count = sheet['properties']['gridProperties']['rowCount']
        col_count = sheet['properties']['gridProperties']['columnCount']

        # Cria um intervalo dinâmico
        SAMPLE_RANGE_NAME = f"principal!A1:{chr(65 + col_count - 1)}{row_count}"

        # Call the Sheets API
        result = (
            service.spreadsheets()
            .values()
            .get(spreadsheetId=SAMPLE_SPREADSHEET_ID, range=SAMPLE_RANGE_NAME)
            .execute()
        )
        values = result.get("values", [])
        
        if not values:
            print("No data found.")
            return []

        return values  # Retorna os valores obtidos

    except HttpError as err:
        print(err)
        return []  # Retorna uma lista vazia em caso de erro

if __name__ == "__main__":
    data = main()


In [21]:
import pandas as pd
df = pd.DataFrame(data)
df

,0,1,2,3,4,5,6,7,8
0,MedInc,HouseAge,AveRooms,AveBedrms,Population,AveOccup,Latitude,Longitude,MedHouseVal
1,"8,3252",41,"6,984126984","1,023809524",322,"2,555555556","37,88","-122,23","4,526"
2,"8,3014",21,"6,238137083","0,9718804921",2401,"2,109841828","37,86","-122,22","3,585"
3,"7,2574",52,"8,288135593","1,073446328",496,"2,802259887","37,85","-122,24","3,521"
4,"5,6431",52,"5,817351598","1,073059361",558,"2,547945205","37,85","-122,25","3,413"
...,...,...,...,...,...,...,...,...,...
20636,"1,5603",25,"5,045454545","1,133333333",845,"2,560606061","39,48","-121,09","0,781"
20637,"2,5568",18,"6,114035088","1,315789474",356,"3,122807018","39,49","-121,21","0,771"
20638,"1,7",17,"5,205542725","1,120092379",1007,"2,325635104","39,43","-121,22","0,923"
20639,"1,8672",18,"5,329512894","1,171919771",741,"2,123209169","39,43","-121,32","0,847"
